In [0]:
# ==============================================================================
# CONFIGURAÇÕES E IMPORTS
# ==============================================================================
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
# Para o XGBoost, a instalação e importação pode variar no Databricks
# Vamos usar o GBTClassifier como um substituto poderoso e nativo do Spark
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import functions as F
import pandas as pd

# Configurar o uso do nosso catálogo e dos schemas
spark.sql("USE CATALOG previsao_brasileirao")
gold_schema = "gold"
diamond_schema = "diamond"

print(f"Lendo dados de: {gold_schema}")
print(f"Salvando modelos e métricas em: {diamond_schema}")

In [0]:
# ==============================================================================
# PASSO 1: CARREGAR A FEATURE STORE E DIVIDIR EM TREINO/TESTE
# ==============================================================================
print("Carregando feature_store e dividindo os dados...")

# Carregar a tabela final da camada Gold
df_features = spark.table(f"{gold_schema}.feature_store")

# Divisão aleatória dos dados
(df_treino, df_teste) = df_features.randomSplit([0.8, 0.2], seed=42)

print(f"Total de partidas: {df_features.count()}")
print(f"Partidas para Treino (80%): {df_treino.count()}")
print(f"Partidas para Teste (20%): {df_teste.count()}")


# --- AJUSTE APLICADO: QUEBRANDO A LINHAGEM DO DATAFRAME DE FORMA DEFINITIVA ---

# 1. Obter os labels distintos como um DataFrame Spark
df_labels_spark = df_features.select("resultado").distinct()

# 2. Coletar os resultados para o driver como uma lista Python.
#    Isso tira os dados do contexto do Spark e quebra qualquer linhagem.
#    A ação .collect() aqui é segura, pois estamos coletando apenas 3 strings.
labels_python_list = [row['resultado'] for row in df_labels_spark.collect()]

# 3. Criar um DataFrame Spark completamente novo e limpo a partir da lista Python.
#    Este DataFrame não tem nenhum plano de execução complexo associado.
df_labels_limpo = spark.createDataFrame(labels_python_list, "string").withColumnRenamed("value", "resultado")

# 4. Treinar (fit) o StringIndexer SOMENTE neste DataFrame limpo.
label_indexer = StringIndexer(inputCol="resultado", outputCol="label")
label_indexer_model = label_indexer.fit(df_labels_limpo)

# 5. Usar o modelo JÁ TREINADO para transformar os DataFrames de treino e teste.
df_treino = label_indexer_model.transform(df_treino)
df_teste = label_indexer_model.transform(df_teste)

# --- FIM DO AJUSTE ---

print("\nAmostra do DataFrame de Treino:")
display(df_treino.limit(5))

In [0]:
# ==============================================================================
# PASSO 2: TREINAR OS MODELOS ESPECIALISTAS (NÍVEL 1)
# ==============================================================================
print("Treinando os 3 modelos especialistas do Nível 1...")

# --- Modelo A: O Generalista (RandomForestClassifier) ---
# Usa todas as features disponíveis
features_gerais_nomes = [col for col in df_treino.columns if col not in ["partida_id", "rodada", "mandante_id", "visitante_id", "resultado", "label"]]

# Converter todas as features para DoubleType para garantir consistência
df_treino_geral = df_treino.select(
    "label",
    *[F.col(c).cast("double").alias(c) for c in features_gerais_nomes]
)

va_geral = VectorAssembler(inputCols=features_gerais_nomes, outputCol="features")

# AJUSTE: Trocado GBTClassifier por RandomForestClassifier, que suporta multiclass
rf_geral = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100, maxDepth=5)

pipeline_geral = Pipeline(stages=[va_geral, rf_geral])
modelo_geral = pipeline_geral.fit(df_treino_geral)
print("  - ✅ Modelo A (Generalista - RandomForest) treinado.")


# --- Modelo B: O Estatístico (Regressão Logística) ---
# Este modelo já suporta multiclass nativamente
features_estatisticas_nomes = ["elo_mandante_pre_jogo", "elo_visitante_pre_jogo", "elo_diff", "mm_gols_m", "mm_gols_v", "diff_mm_gols"]

df_treino_lr = df_treino.select(
    "label",
    *[F.col(c).cast("double").alias(c) for c in features_estatisticas_nomes]
)

va_estatistico = VectorAssembler(inputCols=features_estatisticas_nomes, outputCol="features")
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=20)
pipeline_lr = Pipeline(stages=[va_estatistico, lr])
modelo_lr = pipeline_lr.fit(df_treino_lr)
print("  - ✅ Modelo B (Estatístico - Regressão Logística) treinado.")


# --- Modelo C: O Tático (Random Forest) ---
# Este modelo também já suporta multiclass nativamente
features_taticas_nomes = ["diff_mm_fin", "diff_mm_des"]

df_treino_rf = df_treino.select(
    "label",
    *[F.col(c).cast("double").alias(c) for c in features_taticas_nomes]
)

va_tatico = VectorAssembler(inputCols=features_taticas_nomes, outputCol="features")
rf_tatico = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=50, maxDepth=5)
pipeline_rf = Pipeline(stages=[va_tatico, rf_tatico])
modelo_rf = pipeline_rf.fit(df_treino_rf)
print("  - ✅ Modelo C (Tático - RandomForest) treinado.")

In [0]:
# ==============================================================================
# PASSO 3: GERAR PREVISÕES E AVALIAR CADA ESPECIALISTA INDIVIDUALMENTE
# ==============================================================================
print("Gerando previsões dos especialistas e avaliando cada um...")

# Obter as previsões de cada modelo especialista
pred_geral = modelo_geral.transform(df_teste) 
pred_lr = modelo_lr.transform(df_teste)
pred_rf = modelo_rf.transform(df_teste)

# --- AVALIAÇÃO INDIVIDUAL ---
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

acc_geral = evaluator.evaluate(pred_geral)
acc_lr = evaluator.evaluate(pred_lr)
acc_rf = evaluator.evaluate(pred_rf)

print("\n--- Acurácia dos Modelos Especialistas (Nível 1) ---")
print(f"  - Modelo A (Generalista - RandomForest): {acc_geral:.2%}")
print(f"  - Modelo B (Estatístico - Regressão Log.): {acc_lr:.2%}")
print(f"  - Modelo C (Tático - RandomForest): {acc_rf:.2%}")


# --- PREPARAÇÃO PARA O META-MODELO ---
# Renomear as colunas de probabilidade para evitar conflitos
meta_features_geral = pred_geral.select("partida_id", "label", F.col("probability").alias("prob_geral"))
meta_features_lr = pred_lr.select("partida_id", F.col("probability").alias("prob_lr"))
meta_features_rf = pred_rf.select("partida_id", F.col("probability").alias("prob_rf"))

# Juntar as previsões para formar o DataFrame de treino do Nível 2
df_treino_meta = meta_features_geral.join(meta_features_lr, "partida_id").join(meta_features_rf, "partida_id")

# Criar um único vetor de features para o meta-modelo
assembler_meta = VectorAssembler(inputCols=["prob_geral", "prob_lr", "prob_rf"], outputCol="features")
df_treino_meta = assembler_meta.transform(df_treino_meta)

print("\n✅ Meta-features criadas com sucesso para o Nível 2.")
display(df_treino_meta.limit(5))

In [0]:
# ==============================================================================
# PASSO 4: TREINAR, AVALIAR E SALVAR OS RESULTADOS DO META-MODELO (NÍVEL 2)
# ==============================================================================
print("Treinando e avaliando o meta-modelo final...")

# Usaremos uma Regressão Logística como o "blender" final.
meta_modelo = LogisticRegression(labelCol="label", featuresCol="features", maxIter=20)
modelo_final = meta_modelo.fit(df_treino_meta)

# Fazer as previsões finais no mesmo dataset de teste
previsoes_finais = modelo_final.transform(df_treino_meta)

# Avaliar a performance
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
acuracia = evaluator.evaluate(previsoes_finais)

print(f"\n--- AVALIAÇÃO FINAL DO MODELO HÍBRIDO ---")
print(f"🎯 Acurácia nas últimas 5 rodadas: {acuracia:.2%}")

# Exibir a Matriz de Confusão para uma análise mais detalhada
print("\nMatriz de Confusão:")
display(previsoes_finais.groupBy("label", "prediction").count())


# --- AJUSTE APLICADO AQUI: PERSISTINDO OS RESULTADOS NA CAMADA DIAMOND ---

# Selecionar as colunas mais importantes para a análise de performance
df_resultados_diamond = previsoes_finais.select(
    "partida_id",
    "label",         # O resultado real (0=V_MAND, 1=EMP, 2=V_VIS)
    "prediction",    # A previsão do nosso modelo
    "probability"    # A confiança do modelo em cada uma das 3 classes
)

# Salvar esta tabela de resultados na Camada Diamond
df_resultados_diamond.write \
    .mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{diamond_schema}.previsoes_validadas")

print(f"\n✅ Tabela de previsões e resultados salva com sucesso em '{diamond_schema}.previsoes_validadas'")

# Em um cenário de produção, você também salvaria o objeto do modelo treinado,
# geralmente usando MLflow, que se integra perfeitamente ao Databricks.
# mlflow.spark.log_model(modelo_final, "modelo_previsao_brasileirao")